# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234645225956                   -0.50    8.0         
  2   -7.250407246490       -1.80       -1.40    1.0   7.20ms
  3   -7.251091219392       -3.16       -2.01    1.0   7.36ms
  4   -7.251025126363   +   -4.18       -1.91    2.0   9.30ms
  5   -7.251333335787       -3.51       -2.69    1.0   7.56ms
  6   -7.251338126290       -5.32       -3.15    1.0   7.46ms
  7   -7.251338716376       -6.23       -3.57    2.0   8.56ms
  8   -7.251338789895       -7.13       -4.09    2.0   8.96ms
  9   -7.251338798221       -8.08       -4.64    2.0   8.44ms
 10   -7.251338798655       -9.36       -5.32    1.0   7.67ms
 11   -7.251338798702      -10.33       -5.86    2.0   9.00ms
 12   -7.251338798704      -11.64       -6.17    2.0   9.59ms
 13   -7.251338798704      -12.33       -6.75    1.0   7.93ms
 14   -7.251338798705      -13.17       -7.20    2.0   56.2ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06787679824548642
[ Info: Arnoldi iteration step 2: normres = 0.567647419829514
[ Info: Arnoldi iteration step 3: normres = 0.6561191695460539
[ Info: Arnoldi iteration step 4: normres = 0.17289332581992634
[ Info: Arnoldi iteration step 5: normres = 0.45897344603729606
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (7.94e-03, 3.17e-02, 4.10e-01, 1.66e-01, 1.17e-01)
[ Info: Arnoldi iteration step 6: normres = 0.22851767891823854
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.75e-03, 2.01e-01, 7.03e-02, 7.28e-02, 9.94e-03)
[ Info: Arnoldi iteration step 7: normres = 0.15414824608722374
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.34e-04, 2.72e-02, 6.45e-03, 1.22e-01, 8.66e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10668685945629146
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (6.29e